<!--NAVIGATION-->
<a href="https://colab.research.google.com/github/marcoteran/machinelearning/blob/master/notebooks/01_machinelearning/03_artificialintelligence_nonlinealclassification_complexity_overfitting.ipynb" target="_blank"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Abrir en Colab" title="Abrir y ejecutar en Google Colaboratory"></a>

### Ejemplo de código
# Sesión 02: Clasificación no lineal, complejidad y sobreajuste
## Machine Learning

**Name:** Marco Teran
**E-mail:** marco.teran@usa.edu.co

[Website](http://marcoteran.github.io/),
[Github](https://github.com/marcoteran),
[LinkedIn](https://www.linkedin.com/in/marcoteran/).
___

Definimos primero unas librerías y funciones que vamos a usar a durante la sesión:

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pylab as pl
from sklearn.datasets import make_moons

In [ ]:
# Función para visualizar un conjunto de datos en 2D
def plot_data(X, y):
    y_unique = np.unique(y)
    colors = pl.cm.rainbow(np.linspace(0.0, 1.0, y_unique.size))
    for this_y, color in zip(y_unique, colors):
        this_X = X[y == this_y]
        pl.scatter(this_X[:, 0], this_X[:, 1],  c=color.reshape(1,-1),
                    alpha=0.5, edgecolor='k',
                    label="Class %s" % this_y)
    pl.legend(loc="best")
    pl.title("Data")
    
# Función para visualizar de la superficie de decisión de un clasificador
def plot_decision_region(X, pred_fun):
    min_x = np.min(X[:, 0])
    max_x = np.max(X[:, 0])
    min_y = np.min(X[:, 1])
    max_y = np.max(X[:, 1])
    min_x = min_x - (max_x - min_x) * 0.05
    max_x = max_x + (max_x - min_x) * 0.05
    min_y = min_y - (max_y - min_y) * 0.05
    max_y = max_y + (max_y - min_y) * 0.05
    x_vals = np.linspace(min_x, max_x, 100)
    y_vals = np.linspace(min_y, max_y, 100)
    XX, YY = np.meshgrid(x_vals, y_vals)
    grid_r, grid_c = XX.shape
    ZZ = np.zeros((grid_r, grid_c))
    for i in range(grid_r):
        for j in range(grid_c):
            ZZ[i, j] = pred_fun(XX[i, j], YY[i, j])
    pl.contourf(XX, YY, ZZ, 100, cmap = pl.cm.coolwarm, vmin= -1, vmax=2)
    pl.colorbar()
    pl.xlabel("x")
    pl.ylabel("y")
    
def gen_pred_fun(clf):
    def pred_fun(x1, x2):
        x = np.array([[x1, x2]])
        return clf.predict(x)[0]
    return pred_fun

# Complejidad

Un modelo de clasificación puede ser tan complejo como para aprenderse de memoria el conjunto de entrenamiento. Esta complejidad está determinada por los parámetros internos del modelo. A continuación, observaremos como se comporta esta complejidad usando un modelo clasificación no lineal como lo es  **k-vecinos más cercanos** (*K-nearest neighbors* en inglés)

## Definición del conjunto de datos

Vamos a trabajar con un conjunto de datos artificial (conjunto de datos de juguete). El conjunto es creado usando la funcionalidad `make_moons` de Scikit-Learn [ver más](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_moons.html).
`make_moons` permite introducir algo de ruido sobre las muestras creadas.

`sklearn.datasets.make_moons(n_samples=100, *, shuffle=True, noise=None, random_state=None)`
* **n_samples** Indica el número de muestras para cada grupo (clúster).
* **n_features** El número de características de cada muestra (clústers).
* **centros** El número de centros a generar o la posición fija del centro.
* **noise** Desviación estándar del ruido gaussiano agregado a las muestras que pertenecen al mismo grupo
* **shuffle** Mezcla las muestras
* **random_state** Semilla del generador de muestras aleatorias. Este parámetro garantiza reproducibilidad del conjunto de datos.

Salidas:
* *X:* lista de las muestras generadas `[n_samples, n_features]`
* *y:* lista de etiquetas `[n_samples]`

In [ ]:
X,y = make_moons(n_samples=600, noise=0.3, random_state=0)

In [ ]:
# Mo strar X ~ n_muestras x n_características
# y ~ n_muestras
# Primeras 5 muestras
# Primeras 5 etiquetas

print('Mostrar el tamaño de X:', X.shape)
print('Mostrar el tamaño de y:', y.shape)

print('Primeras 5 muestras\n', X[:5,:])
print('Primeras 5 etiquetas\n', y[:5])

In [ ]:
# Dibujar los datos
pl.figure(figsize=(10,6))
plot_data(X,y)

Observamos que es dificil establecer una separación lineal como en regresión logística. Por lo tanto es necesario usar un modelo de clasificación no lineal.

# Algoritmo: K-vecinos más cercanos

La clasificación basada en vecinos es un tipo de aprendizaje basado en ejemplos. El modelo almacena los ejemplos vistos durante entrenamiento y clasifica un elemento no visto, usando una simple regla de votación por mayoría. Si se ubica un **punto** en el espacio de características, se le asigna como clase el valor de la clase que tenga la mayor cantidad de ejemplos en la vecindad del punto. Este ejemplo lo podemos ver ilustrado en la imagen:

<img src="https://github.com/marcoteran/artificialintelligence/raw/master/notebooks/01_machinelearnig/figures/knn.png" width="50%">

Scikit-Learn provee una implementación del algoritmo conocida como `KNeighborsClassifier`. `KNeighborsClassifier` tiene un parámetro $n\_neighbors$ o $k$, dónde $k$ es un entero definido por el usuario que determina cuantos vecinos evalua para determinar la clase de una instancia nunca antes vista. La elección de este parámetro es definida totalmente por la naturaleza de los datos.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

La librería te ofrece varios parámetros que puedes utilizar para configurar el algoritmo y algunos de ellos son muy útiles para mejorar el modelo que se este construyendo.

* El primer parámetro, por su puesto debe ser el del número de vecinos o $k$, acá es donde defines este valor, a este parámetro se le conoce acá como `n_neighbors` (por defecto es $5$). Observaremos que dependiendo del valor de vecinos más cercanos, conseguimos diferentes funciones de ajuste, unas más suaves que otras. Vamos a evaluar el efecto del parámetro $k$ en la complejidad del modelo.
* Otro parámetro importante es definir la distancia que se utilizará para verificar los vecinos del dato que se está buscando predecir. Para configurar esto en el algoritmo se debe definir dos variables dentro del algoritmo, la primera es `p` y la segunda es `metric`:
    - `p` Es la función que computa la distancia, podemos elegir distintas, Euclidean con P=2, Manhatan con P=1, Mikowski cualquier otro valor.
    - `metric` Variar distancia Mikowski a otras más complejas
* weights: La ponderación de los votos de cada punto, puede ser:
    - uniforme: todos los puntos valen lo mismo.
    - distance: los votos de los puntos más cercanos tienen más valor
* algorithm: controla el algoritmo que computa las distancias.

In [ ]:
knn=KNeighborsClassifier(n_neighbors=1)

Vamos a extraer un porcentaje al azar del conjunto de datos. 

In [ ]:
# X_reduced, y_reduced al 50% random
idx=np.random.choice(len(X), int(len(X)*0.5), replace=False)
X_reduced=X[idx]
y_reduced=y[idx]

Entrenamos un modelo `knn` llamando la función `fit()` sobre el conjunto de datos reducido

In [ ]:
knn.fit(X_reduced, y_reduced)

In [ ]:
# Dibujar la superficie de decición
pl.figure(figsize=(10,6))
plot_decision_region(X_reduced,gen_pred_fun(knn))
plot_data(X_reduced, y_reduced)

In [ ]:
# Mostrar el error
print(knn.score(X_reduced, y_reduced))
print('Error: {}'.format(1-knn.score(X_reduced, y_reduced)))

**¿Tiene sentido que el valor del error?**

Ahora agreguemos los datos que descartamos anteriormente

In [ ]:
# X_complement, y_complement
idx_c = [i for i in range(len(X)) if i not in idx]

X_complement=X[idx_c]
y_complement=y[idx_c]

In [ ]:
# Dibujemos la superficie de decisión
pl.figure(figsize=(10,6))
plot_decision_region(X_complement,gen_pred_fun(knn))
plot_data(X_complement, y_complement)

In [ ]:
# Mostrar el error
print(knn.score(X_complement, y_complement))
print('Error: {}'.format(1-knn.score(X_complement, y_complement)))

Podemos observar que cuando el número de vecinos es $1$, el modelo se ajusta demasiado al ruído de los datos de entrada, por lo tanto sufre de **sobreajuste**.

# Error de entrenamiento y generalización

Un modelo de aprendizaje de máquina tiene como objetivo principal hacer predicciones de manera acertada sobre ejemplos nunca antes vistos por el modelo. Esto se conoce como error de generalización. Para poder medir el error de generalización, dividimos el conjunto de datos en dos particiones: 

* **Entrenamiento**: Se usará para entrenar el modelo.
* **Prueba**: Se usará para medir el error de generalización.

En la siguiente imagen encontramos una ilustración de cómo se hace un particionamiento en entrenamiento y prueba.

<img src="https://github.com/marcoteran/artificialintelligence/raw/master/notebooks/01_machinelearnig/figures/train_test_split.svg" width="50%">

Una de las prácticas recomendadas, es particionar los datos $70\%$ para entrenamiento y $30\%$ para prueba. Cuando el número de muestras es muy grande ($\ge 70K$), podemos reducir el porcentaje de muestras para prueba, a $90-10\%$. Sin embargo, deben hacerse unas aclaraciones sobre la generalización:

* El conjunto de prueba debe ser una muestra representativa del conjunto de datos. El muestreo de ejemplos debe hacerse de forma independiente e idénticamente aleatoria de una distribución. Esto quiere decir, que el muestreo de un ejemplo no está influenciado por el muestreo de otro.
* La distribución es estacionaria. Es decir no cambia a lo largo del conjunto de datos.
* Los ejemplos son muestreados desde particiones de la misma distribución. Es decir, no se deben crear nuevas características en la partición de prueba.

Adicionalmente, debemos tener en cuenta que se conserve la distribución de las etiquetas de los datos tanto en entrenamiento como en prueba (estratificación). En la siguiente sesión se va a estudiar en más detalle los efectos de hacer una partición estratificada. Scikit-Learn nos permite particionar un conjunto de datos en entrenamiento y prueba. A continuación vamos a dividir el conjunto en $70\%$ para entrenamiento y $30\%$ para prueba.

In [ ]:
from sklearn.model_selection import train_test_split

#### Parámetros:

* `test_size`: Tamaño de la partición de prueba
* `random_state`: Semilla del generador de números pseudoaleatorios. Este parámetro garantiza reproducibilidad del particionamiento.
* `stratify`: Si se estratifican los datos con respecto a `y`

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,
                                                     test_size=0.3,
                                                     stratify=y)

Vamos a verificar el número de muestras de ambas particiones y la distribución de clases de cada una.

In [ ]:
# Mostrar
# Número de muestras en entrenamiento
# Número de muestras en prueba
# Número de características
# Distribución de clases en entrenamiento
# Distribución de clases en prueba
print(X_train.shape[0])
print(X_test.shape[0])
print(X_train.shape[1])

print(np.bincount(y_train))
print(np.bincount(y_test))

Usamos la partición recién creada y analizamos un modelo entrenado con $k = 200$.

In [ ]:
knn=KNeighborsClassifier(n_neighbors=200)
knn.fit(X_train, y_train)

In [ ]:
# Dibujar superficie de decición
pl.figure(figsize=(10,6))
plot_decision_region(X_train,gen_pred_fun(knn))
plot_data(X_train, y_train)

In [ ]:
# Mostrar el error de entrenamiento
print('Error: {}'.format(1-knn.score(X_train, y_train)))

Observamos que el error en entrenamiento es del $14\%$. Además se evidencia que el modelo entrenado es ahora demasiado **simple** y no se puede ajustar la estructura de los datos.

Ahora medimos el error de generalización del modelo entrenado y visualizamos la clasificación de los datos de prueba.

In [ ]:
# Dibujar superficie de decición con los datos de prueba
pl.figure(figsize=(10,6))
plot_decision_region(X_test,gen_pred_fun(knn))
plot_data(X_test, y_test)

In [ ]:
# Mostrar el error de generalización
print('Error: {}'.format(1-knn.score(X_test, y_test)))

Podemos observar que cuando aumentamos el número de vecinos, nuestro modelo sufre de **subajuste**. La superficie de decisión se suaviza, pero no logra captar los detalles de los datos. Mientras el error de entrenamiento se acerca a $14\%$, el error de generalización se acerca a $10\%$.

**¿Cómo estimar un buen número de $k$-vecinos más cercanos de manera que el modelo no sobreajuste ni subajuste los datos?**

# Evaluación de la complejidad

Un modelo de aprendizaje de máquina puede ser tan complejo como para recordar las particularidades y el ruido del conjunto de entrenamiento (**sobreajuste**), así como puede ser demasiado flexible para no modelar la variabilidad de los datos (**subajuste**). El modelo debe garantizar un compromiso entre el sobreajuste y el subajuste, lo cual se logra evaluando la complejidad del modelo. Una forma de evaluar la complejidad es analizar el error de entrenamiento y generalización para diferentes modelos que varían en su complejidad. En el caso de `KNearestNeighbor`, la complejidad está determinada por el número de vecinos $k$. **Entre menor sea el número de vecinos, más complejo es el modelo.**

A continuación exploramos un conjunto de valores $k$, con el objetivo de encontrar aquel modelo con el mejor compromiso entre error de entrenamiento y error de generalización.

In [ ]:
k_values = list(range(1, 20))
print(k_values)

Vamos a generar un nuevo conjunto de datos con $1000$ muestras y haremos una partición $70-30$.

In [ ]:
X,y = make_moons(n_samples=1000, noise=0.4, random_state=0)

X_train, X_test, y_train, y_test = train_test_split(X,y,
                                                     test_size=0.3,
                                                     stratify=y)

Guardamos el error de entrenamiento y generalización con respecto al aumento de complejidad del modelo.

In [ ]:
train_error = []
generalization_error = []

# Utilizar un for

for nn in k_values:
  knn = KNeighborsClassifier(n_neighbors=nn)
  knn.fit(X_train, y_train)
  train_error.append(1-knn.score(X_train, y_train))
  generalization_error.append(1-knn.score(X_test, y_test))

Visualizamos ambas curvas de aprendizaje.

In [ ]:
# Visualizar las curvas de aprendizaje
pl.figure(figsize=(10,6))

pl.plot(k_values, train_error, label='error de entrenamiento')
pl.plot(k_values, generalization_error, label='error de generalización')
pl.xticks(k_values)
pl.xlabel("k-vecinos")
pl.ylabel("Error")
pl.legend()

Encontramos que el error de entrenamiento y generalización tiene su punto de balance mínimo con $k=13$. Observamos tambien que cuando el modelo es demasiado complejo ($k=1$), el error de generalización sube, así como el de entrenamiento cae a $0\%$.